In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import tqdm

d:\Anaconda\envs\sklearn-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_data = []

with open("D:/allforwork/AI/nnsp/NS-2025-07-data/test_input.json") as fi:
    test_data = json.load(fi)

print(test_data[:3])

["The behavior tree orchestrates a robot's path planning and execution. It first computes a path using the global planner, then runs two sub-branches in parallel. The first sub-branch involves executing the local planner to follow the computed path, while the second sub-branch controls the rate at which new paths are computed by the global planner. The goal for the global planner and the resulting path are stored on the blackboard for communication between different components.\n\nThe RateController node regulates the frequency of invoking the ComputePathToPose operation. The Parallel node uses a threshold of 1 to return the result of FollowPath whenever it returns SUCCESS or FAILURE. This ensures that the overall task status reflects the success or failure of the path following operation.\n\nIn summary, the behavior tree manages the coordination of global and local path planning and execution for the robot, ensuring continuous path updates and adaptive control based on the robot's pro

In [3]:
model_name = "Qwen/Qwen2_5-1_5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_mode=True)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda").eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
prompt = "按照以下要求完成将自然语言转换为结构化的 XML 格式行为树的任务：\n1.生成合法、可解析的XML代码\n使用标准行为树标签（如 <BehaviorTree>, <Sequence>, <Action> 等）\n3.符合 Groot 可视化工具的兼容要求\n4.直接输出且只需输出最终的XML代码，无需解释\n5.给出的自然语言指令为英语\n6.确保代码实现了指令中的所有功能\n英语自然语言指令："
prompt = "你是一个将自然语言指令转换为XML格式行为树的专家，下面给一段英文自然语言指令，请你\n1.**仔细分析、识别并理解**指令中的所有逻辑和功能\n2.然后**一一实现**指令中的所有逻辑和功能\n3.生成对应结构化的XML格式行为树\n自然语言指令："
for i in range(len(test_data)):
    test_data[i] = prompt + test_data[i] + "\n对应的XML格式行为树代码(直接生成代码，无需解释)："
test_data[0]

"你是一个将自然语言指令转换为XML格式行为树的专家，下面给一段英文自然语言指令，请你\n1.**仔细分析、识别并理解**指令中的所有逻辑和功能\n2.然后**一一实现**指令中的所有逻辑和功能\n3.生成对应结构化的XML格式行为树\n自然语言指令：The behavior tree orchestrates a robot's path planning and execution. It first computes a path using the global planner, then runs two sub-branches in parallel. The first sub-branch involves executing the local planner to follow the computed path, while the second sub-branch controls the rate at which new paths are computed by the global planner. The goal for the global planner and the resulting path are stored on the blackboard for communication between different components.\n\nThe RateController node regulates the frequency of invoking the ComputePathToPose operation. The Parallel node uses a threshold of 1 to return the result of FollowPath whenever it returns SUCCESS or FAILURE. This ensures that the overall task status reflects the success or failure of the path following operation.\n\nIn summary, the behavior tree manages the coordination of global and loc

In [5]:
results = []
for que in tqdm.tqdm(test_data):
    messages = [
        {"role": "system", "content": "你是一个将自然语言指令转换为XML格式行为树的专家，根据以下要求完成转换任务：\n1.生成合法、可解析的XML代码\n2.符合BehaviorTree.CPP要求，使用标准行为树标签（如 <BehaviorTree>, <Sequence>, <Action> 等）\n3.符合 Groot 可视化工具的兼容要求\n4.确保代码实现了指令中的**所有**逻辑和功能\n"},
        {"role": "user", "content": que}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=5000
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    results.append(response)

100%|██████████| 119/119 [13:15<00:00,  6.68s/it]


In [21]:
results

['```xml\n<BehaviorTree>\n    <Sequence>\n        <Node name="Compute Path">\n            <Action name="Global Planner" />\n        </Node>\n        <Node name="Parallel">\n            <Threshold name="SuccessOrFailure" value="1" />\n            <Action name="Follow Path" />\n        </Node>\n        <Node name="Rate Controller">\n            <Action name="Invoke Frequency" />\n        </Node>\n    </Sequence>\n</BehaviorTree>\n```']

In [7]:
infer_pipeline = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    device=0
)

Device set to use cuda:0


In [8]:
results = infer_pipeline(test_data, batch_size=1)

In [6]:
results[10]

'```xml\n<BehaviorTree>\n  <Sequence name="BatteryCheck">\n    <Action name="CheckBattery" />\n    <Condition name="BatteryNeedsRecharge">\n      <If condition="batteryLevel <= threshold">\n        <Action name="NavigateToRechargeStation" />\n        <Action name="InitiateRechargeProcess" />\n      </If>\n    </Condition>\n  </Sequence>\n\n  <Sequence name="PatrolMode">\n    <Action name="EnterPatrolMode" />\n    <Repeat name="PatrolThroughWaypoints">\n      <Action name="NavigateToNextWaypoint" />\n    </Repeat>\n  </Sequence>\n\n  <Parallel name="MainLoop">\n    <Sequence name="BatteryCheck">\n      <Sequence name="PatrolMode">\n        <Branch>\n          <If condition="batteryLevel > threshold">\n            <Action name="ContinuePatrol" />\n          </If>\n          <Else>\n            <Action name="ChargeRobot" />\n          </Else>\n        </Branch>\n      </Sequence>\n    </Sequence>\n  </Parallel>\n</BehaviorTree>\n```'

In [7]:
results[118]

'```xml\n<BehaviorTree>\n    <RootNode name="NavigationWithReplanning">\n        <Sequence>\n            <Action name="NavigateWithReplanning" />\n            <IfCondition name="CheckForFailures">\n                <Action name="CheckForFailures" />\n                <Then>\n                    <Action name="Retry" />\n                </Then>\n                <Else>\n                    <Action name="ClearLocalAndGlobalCostmaps" />\n                    <Action name="SpinAction" />\n                    <Action name="WaitDuration" />\n                </Else>\n            </IfCondition>\n        </Sequence>\n        <RecoveryFallback name="RecoveryFallback">\n            <Sequence>\n                <Action name="ClearBothCostmaps" />\n                <Action name="PerformSpinAction" />\n                <Action name="WaitDuration" />\n            </Sequence>\n        </RecoveryFallback>\n    </RootNode>\n</BehaviorTree>\n```'

In [8]:
with open("D:/allforwork/AI/nnsp/NS-2025-07-data/results.xml", "w") as fi:
    for i in range(len(results)):
        ans = results[i][6:-3]
        print(f'  <item id="{i + 1}"><![CDATA[{ans}  ]]></item>\n', file=fi)